# 3 图像几何变换

## 3.1 图像几何变换的基本理论

### 3.1.1 图像几何变换概述

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;图像几何变换用于改变图像像素与像素之间的空间关系，从而重构图像达到图像处理的目的。简而言之就是建立源图像像素和输出图像像素之间的映射关系。<br>

<center>$\left\{_{y=V(x_0,y_0)}^{x=U(x_0,y_0)} \right.$</center>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;其中： $x_0，y_0$ 表示原图像像素坐标，x,y是输出图像像素坐标。坐标变换可以是线性映射，如：
<center>$\left\{_{V(x_0,y_0)=k_4x_0+k_5y_0+k_6}^{U(x_0,y_0)=k_1x_0+k_2y_0+k_3} \right.$</center>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;也可以是非线性变换，如：
<center>$\left\{_{V(x_0,y_0)=k_7+k_8x_0+k_9y_0+k_{10}x^2+k_{11}xy+k_{12}y^2}^{U(x_0,y_0)=k_1+k_2x_0+k_3y_0+k_4x^2+k_5xy+k_6y^2} \right.$</center>
注：几何变换本质上是像素值不变，像素的位置变换，造成图像几何变换的效果。<br>
变换实现方法：构建一个与空白目标图像，将原图像中的各个像素的值依次按照变换函数的计算结果复制到相应的像素位置。

#### 一、向前映射

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;由源图像坐标推算出输出图像坐标称为向前映射。
<img src="imgs/向前映射.png" width="40%">
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;存在的问题：<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1、浮点坐标。例如原图像像素（3，3）经过映射变换后映射为(1.5,1.5)。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2、映射不完全或者映射重叠。<br>
举例：<br>
图像放大（映射不完全）
<img src="imgs/映射不完全.png" width="40%">
图像缩小（映射重叠）
<img src="imgs/映射重叠.png" width="40%">

#### 二、向后映射

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;向后映射是指用输出图像中元素坐标(x,y)就是将输出图像$f(x,y)$遍历一遍，然后计算输出$(x,y)$的时候，所需要$g(x,y)$的坐标。当然，这个数不一定是整数。如果不是整数，则需要通过插值的方式计算像素值。<br>
<img src="imgs/interBack.png" width="40%">

### 3.2 图像平移变换

基本原理：
<center>$\left\{_{y=y_0+dy}^{x=x_0+dx} \right.$</center>
矩阵表示法：<br>
<center>$\left[ \begin{array} {cccc}
x &y&1 
\end{array} \right]
=\left[ \begin{array} {cccc}
x_0 &y_0&1 
\end{array} \right]\left[ \begin{array} {cccc}
1&0&0\\
0&1&0\\
dx&dy&1
\end{array} \right]$</center>
根据上述公式或者矩阵可以计算出图像平移后各个像素的新位置，新位置坐标可能出现的后果：<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(1)新位置越界到图像外面。处理办法：适当扩大图像或者直接丢弃。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(2)像素移走后空余出的区域补0或255.

In [2]:
#代码实现--不使用Opencv函数
import cv2
import numpy as np
img = cv2.imread("C:\\Users\\Administrator\\ImageProcess\\images\\jeep1.jpeg", 1)
imgInfo = img.shape
height = imgInfo[0]
width = imgInfo[1]
mode = imgInfo[2]

dst = np.zeros(imgInfo, np.uint8)

for i in range( height ):
    for j in range( width - 100 ):
        dst[i, j + 100] = img[i, j]
cv2.imshow('src', img)
cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
#代码实现--使用Opencv函数
import numpy as np
import cv2

img = cv2.imread("C:\\Users\\lenovo\\ImageProcess\\road.jpg")
cv2.imshow("src", img)
img_info=img.shape
height=img_info[0]
width=img_info[1]
mat_translation=np.float32([[1,0,20],[0,1,50]])  #变换矩阵：设置平移变换所需的计算矩阵：2行3列
#[[1,0,20],[0,1,50]]   表示平移变换：其中20表示水平方向上的平移距离，50表示竖直方向上的平移距离。
dst=cv2.warpAffine(img,mat_translation,(width+20,height+50))  #变换函数
'''
参数2 变换矩阵：是一个2行3列的矩阵，由这个矩阵决定是何种变换
参数3 变换后输出图像的大小:(width+20,height+50)-->宽和高(自己规定)
参数4 可选参数，用于设置插值方法的组合，默认为INTER_LINEAR使用线性插值算法，除了图像缩放中用到的几个插值算法外，仿射变换还可以选用
INTER_LANCZOS4（Lanczos插值算法）。
参数5 borderValue：可选参数，在边界不变的情况下可以使用的值，主要用于设置边界的填充值，默认为0
'''
cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 3.3 图像镜像变换

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;图像镜像变换分为水平镜像和垂直镜像。水平镜像是图像的左右两部分对调，垂直变换是上下两部分对调。<br>
图像镜像的变换公式:<br>
1、水平镜像的变换关系<br>
<center>$\left[ \begin{array} {cccc}
x &y&1 
\end{array} \right]
=\left[ \begin{array} {cccc}
x_0 &y_0&1 
\end{array} \right]\left[ \begin{array} {cccc}
-1&0&0\\
0&1&0\\
Width&0&1
\end{array} \right]=\left[ \begin{array} {cccc}
Width-x_0&y_0&1 
\end{array} \right]$</center><br>
对矩阵求逆，得到<br>
<center>$\left[ \begin{array} {cccc}
x_0 &y_0&1 
\end{array} \right]
=\left[ \begin{array} {cccc}
x &y&1 
\end{array} \right]\left[ \begin{array} {cccc}
-1&0&0\\
0&1&0\\
Width&0&1
\end{array} \right]=\left[ \begin{array} {cccc}
Width-x&y&1 
\end{array} \right]$</center><br>
2、垂直镜像的变换关系<br>
<center>$\left[ \begin{array} {cccc}
x &y&1 
\end{array} \right]
=\left[ \begin{array} {cccc}
x_0 &y_0&1 
\end{array} \right]\left[ \begin{array} {cccc}
1&0&0\\
0&-1&0\\
0&Height&1
\end{array} \right]=\left[ \begin{array} {cccc}
x_0&Height-y_0&1 
\end{array} \right]$</center><br>
对矩阵求逆，得到<br>
<center>$\left[ \begin{array} {cccc}
x_0 &y_0&1 
\end{array} \right]
=\left[ \begin{array} {cccc}
x &y&1 
\end{array} \right]\left[ \begin{array} {cccc}
1&0&0\\
0&-1&0\\
0&Height&1
\end{array} \right]=\left[ \begin{array} {cccc}
x&Height-y&1 
\end{array} \right]$</center><br>


In [6]:
#垂直镜像的代码实现--不使用Opencv函数
#水平镜像自己编码实现
import cv2
import numpy as np
img = cv2.imread('C:\\Users\\lenovo\\ImageProcess\\road.jpg', 1)
dim = (320, 240)
# resize image
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
cv2.imshow('src', img)
imgInfo = img.shape
height= imgInfo[0]
width = imgInfo[1]
deep = imgInfo[2]

dst = np.zeros([height, width, deep], np.uint8)

for i in range( height ):
    for j in range( width ):
        #dst[i,j] = img[i,j]
        dst[height-i-1,j] = img[i,j]

#for i in range(width):
#    dst[height, i] = (0, 0, 255)
cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
#垂直镜像的代码实现--使用Opencv函数
#水平镜像自己编码实现
import numpy as np
import cv2

img = cv2.imread("C:\\Users\\lenovo\\ImageProcess\\road.jpg")
cv2.imshow("src", img)
img1 = cv2.flip(img,1)  #镜像
'''
参数2 必选参数。用于指定镜像翻转的类型，其中0表示绕×轴正直翻转，即垂直镜像翻转；1表示绕y轴翻转，即水平镜像翻转；-1表示绕×轴、y轴两个轴翻转，即对角镜像翻转。
参数3 可选参数。用于设置输出数组，即镜像翻转后的图像数据，默认为与输入图像数组大小和类型都相同的数组。
'''

cv2.imshow('dst',img1)

cv2.waitKey(0)
cv2.destroyAllWindows()

### 3.4 图像的缩放

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;假设图像x轴方向的缩放比例为$S_x$,y轴缩放比例为$S_y$,相应的变换公式为：<br>
<center>$\left [ \begin{array} {cccc}
x &y&1 
\end{array} \right]
=\left[ \begin{array} {cccc}
x_0 &y_0&1 
\end{array} \right]\left[ \begin{array} {cccc}
S_x&0&0\\
0&S_y&0\\
0&0&1
\end{array} \right]=\left[ \begin{array} {cccc}
x_0*S_x&y_0*S_y&1 
\end{array} \right]$</center><br>
其逆运算如下：<br>
<center>$\left [ \begin{array} {cccc}
x_0 &y_0&1 
\end{array} \right]
=\left[ \begin{array} {cccc}
x &y&1 
\end{array} \right]\left[ \begin{array} {cccc}
\frac{1}{S_x}&0&0\\
0&\frac{1}{S_y}&0\\
0&0&1
\end{array} \right]=\left[ \begin{array} {cccc}
\frac{x}{S_x}&\frac{y}{S_y}&1 
\end{array} \right]$</center><br>


In [9]:
#代码实现
import cv2 as cv
 
# 读入原图片
img = cv.imread('C:\\Users\\lenovo\\ImageProcess\\road.jpg')
# 打印出图片尺寸
print(img.shape)
# 将图片高和宽分别赋值给x，y
x, y = img.shape[0:2]
 
# 显示原图
cv.imshow('OriginalPicture', img)
 
# 缩放到原来的二分之一，输出尺寸格式为（宽，高）
img_test1 = cv.resize(img, (int(y / 2), int(x / 2)))
cv.imshow('resize0', img_test1)
cv.waitKey()
 
# 最近邻插值法缩放
# 缩放到原来的四分之一
img_test2 = cv.resize(img, (0, 0), fx=0.25, fy=0.25, interpolation=cv.INTER_NEAREST)
cv.imshow('resize1', img_test2)
cv.waitKey()
cv.destroyAllWindows()

(653, 994, 3)


### 3.6 图像旋转

In [10]:
# -*- coding:utf-8 -*-
from math import *
import cv2
import numpy as np

def rotate_bound1(image, angle):
    '''
     . 旋转图片
     . @param image    opencv读取后的图像
     . @param angle    (逆)旋转角度
    '''
    # img = cv2.imread("img/1.jpg")
    (h, w) = image.shape[:2]  # 返回(高,宽,色彩通道数),此处取前两个值返回
    # 抓取旋转矩阵(应用角度的负值顺时针旋转)。参数1为旋转中心点;参数2为旋转角度,正的值表示逆时针旋转;参数3为各向同性的比例因子
    M = cv2.getRotationMatrix2D((w / 2, h / 2), -angle, 1.0)
    # 计算图像的新边界维数
    newW = int((h * np.abs(M[0, 1])) + (w * np.abs(M[0, 0])))
    newH = int((h * np.abs(M[0, 0])) + (w * np.abs(M[0, 1])))
    # 调整旋转矩阵以考虑平移
    M[0, 2] += (newW - w) / 2
    M[1, 2] += (newH - h) / 2
    # 执行实际的旋转并返回图像
    return cv2.warpAffine(image, M, (newW, newH)) # borderValue 缺省，默认是黑色


def rotate_bound2(image, angle):    
    '''
     . 旋转图片
     . @param image    opencv读取后的图像
     . @param angle    (逆)旋转角度
    '''

    h, w = image.shape[:2]  # 返回(高,宽,色彩通道数),此处取前两个值返回
    newW = int(h * fabs(sin(radians(angle))) + w * fabs(cos(radians(angle))))
    newH = int(w * fabs(sin(radians(angle))) + h * fabs(cos(radians(angle))))
    M = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
    M[0, 2] += (newW - w) / 2
    M[1, 2] += (newH - h) / 2
    return cv2.warpAffine(image, M, (newW, newH), borderValue=(255, 255, 255))


image = cv2.imread('C:\\Users\\lenovo\\ImageProcess\\road.jpg')
img = rotate_bound1(image, 60)
img2 = rotate_bound2(image, 60)
cv2.imshow('ww1', img)
cv2.imshow('ww2', img2)
cv2.waitKey()
cv2.destroyAllWindows()

## 3.7 插值算法

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;实现几何运算时，有两种方法。第一种称为向前映射法，其原理是将输入图像的灰度一个像素一个像素地转移到输出图像中，即从原图像坐标计算出目标图像坐标: $g(x, y)=f(a(x_0,y_0),b(x_0,y_0))$。前面的平移、镜像等操作就可以采用这种方法。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;另外一种称为向后映射法,它是向前映射变换的道，即输出像素一个一个地映射回输入。如果一个输出像素映射到的不是输入图像的采样栅格的整数坐标处的像素点,则其灰度值就需要于整数坐标的灰度值进行推断,这就是插值。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 由于向后映射法是逐个像素产生输出图像，不会产生计算浪费问题，所以在缩放、旋转等操作中多采用这种方法，本书中采用的也全部为向后映射法。<br>
本节中将介绍3种不同的插值算法，处理效果好的算法-般需要较大的计算量。<br>
<img src="imgs/向后映射.png" width="40%">

### 3.7.1 最近邻插值
这是一种最简单的插值算法，输出像素的值为输入图像中与其最邻近的采样点的像素值。例如：输出图像的某元素计算出的坐标不是整数，如何取原始图像中的元素值，如下图：<br>
<img src="imgs/最近邻插值.png" width="40%">
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;最近邻的思想就是取计算坐标距离原始图像中最近元素的灰度值。通过计算与点P(x0,y0）临近的四个点，并将与点P(x0,y0）最近的整数坐标点（x,y）的灰度值取为P(x0,y0）点灰度近似值。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;优点：当P(x0,y0）点各相邻像素间灰度变化较小时，此法是最简单快速的插值方法。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;缺点：当P(x0,y0）点相邻像素间灰度值差异很大时，该灰度估值方法会产生较大的误差，甚至可能影响图像质量。

### 3.7.2 双线性插值
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;双线性插值，又称为双线性内插。在数学上，双线性插值是有两个变量的插值函数的线性插值扩展，其核心思想是在两个方向分别进行一次线性插值。双线性插值作为数值分析中的一种插值算法，广泛应用在信号处理，数字图像和视频处理等方面。<br>
举例：<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;已知的红色数据点与待插值得到的绿色点
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;假如我们想得到未知函数$f$在点$P= (x,y)$ 的值，假设我们已知函数$f$在$Q11 = (x1,y1)、Q12 = (x1,y2),Q21 = (x2,y1)$ 以及$Q22 = (x2,y2)$ 四个点的值。首先在x方向进行线性插值，得到$R1$和$R2$，然后在y方向进行线性插值，得到$P$。这样就得到所要的结果$f(x,y)$。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;其中红色点$Q11,Q12,Q21,Q22$为已知的4个像素点。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;第一步：X方向的线性插值，在$Q12,Q22$中插入蓝色点$R2，Q11，Q21$中插入蓝色点R1； <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;第二步 ：Y方向的线性插值 ,通过第一步计算出的R1与R2在y方向上插值计算出P点。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;线性插值的结果与插值的顺序无关。首先进行y方向的插值，然后进行x方向的插值，所得到的结果是一样的。双线性插值的结果与先进行哪个方向的插值无关。<br>
<img src="imgs/双线性插值.jpg" width="40%">

### 3.7.3 高阶插值
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;在一些几何运算中， 双线性插值的平滑作用会使图像的细节退化， 而其斜率的不连续性则会导致变换产生不希望的结果。这些都可以通过高阶插值得到弥补，高阶插值常用卷和来实现，输出像素的值为输入图像中距离它最近的4*4领域内来样点像素值的加极平均值。下面以三次插值为例，它使用了如下的三次多项式来迫近理论上的最佳插值函数。三次插值用理论计算出的坐标邻域的16个点加权组合成一个像素值，关键问题是如何构建各个邻域元素的权值。<br>
    <img src="imgs/三次插值.png" width="40%">
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;构建Bicubic函数，以函数值作为像素的权值。<br>
<img src="imgs/BiCubic.png" width="40%">
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;其中$|x|$是元素距离理论计算点的距离。距离越远权值越小。


# 3.8 OpenCV 中几何变换相关的几个函数

### 3.8.1 缩放
OpenCv中用cv2.resize()函数实现缩放。该函数具体形式如下：<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;cv2.resize(InputArray src, OutputArray dst, Size, fx, fy, interpolation)<br>
参数解释：<br>
<img src="imgs/resizepara.png" width="40%">

In [11]:
#代码举例
import cv2
import numpy as np
img=cv2.imread('imgs\\lena.bmp')
rows,cols=img.shape[:2]
size=(int(cols*0.6),int(rows*0.7))
rst_INTER_AREA=cv2.resize(img,size,cv2.INTER_AREA)
rst_INTER_CUBIC=cv2.resize(img,size,cv2.INTER_CUBIC)
rst_INTER_LINEAR=cv2.resize(img,size,cv2.INTER_LINEAR)
cv2.imshow('INTER_AREA', rst_INTER_AREA)
cv2.imshow('rst_INTER_CUBIC', rst_INTER_CUBIC)
cv2.imshow('rst_INTER_LINEAR', rst_INTER_LINEAR)
cv2.imshow('src', img)
cv2.waitKey()
cv2.destroyAllWindows()

### 3.8.2 反转
OpenCv使用cv2.flip()函数实现反转。格式如下：<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;dst=cv2.flip(src, flipCode[, dst])<br>
参数解释：(1)dst:目标图像；（2）src原图像；（3）flipcode代表旋转类型。0：绕x轴，其余正整数：绕y轴旋转。负整数：同时绕两轴。

In [ ]:
#代码举例
import cv2
import numpy as np
img=cv2.imread('imgs\\lena.bmp')
dst_x=cv2.flip(img,0)
dst_y=cv2.flip(img,2)
dst_xy=cv2.flip(img,-1)
cv2.imshow('flip_x', dst_x)
cv2.imshow('flip_y', dst_y)
cv2.imshow('flip_xy',dst_xy)
cv2.imshow('src', img)
cv2.waitKey()
cv2.destroyAllWindows()

### 3.8.3 仿射变换
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;仿射变换是指图像可以通过一系列几何变换来实现平移、旋转等操作。仿射变换能够保持图像的平行性和平直性。平行性是指原图中平行的直线变换后仍然是平行的。平直性是指变换后直线仍然是直线。<br>
  OpenCV 中的仿射函数为cv2.warpAffine(),其通过一个变换矩阵M实现变换，具体为：<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; <center>$dst(x,y)=src(M_{11}x+M_{12}y+M_{13},M_{21}x+M_{22}y+M_{23}$</center><br>
 以上公式与前述平移、反转等操作相同。<br>
 cv2.warpAffine()的语法如下：<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;dst=cv2.warpAffine(src, M, dsize[, dst[, flags[, borderMode[, borderValue]]]])<br>
参数定义：<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;src: 输入图像。<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;dst: 输出图像，尺寸由dsize指定，图像类型与原图像一致。<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;M: 2X3的变换矩阵,不同的变换矩阵对应着不同的图像变换。<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;dsize: 指定图像输出尺寸。<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;flags: 插值算法标识符，有默认值INTER_LINEAR，如果插值算法为WARP_INVERSE_MAP, warpAffine函数使用如下矩阵进行图像转换。<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;borderMode: 边界像素模式，有默认值BORDER_CONSTANT。<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;borderValue: 边界取值(经过变换后空余出来的区域填充什么值)，有默认值Scalar()即0。<br>
 
 特别说明：仿射变换最核心的内容是构造变换矩阵。如果变换矩阵已知可以直接用nump定义，如果不易知道则可以利用opencv提供的函数生成。

##### 3.8.3.1 仿射变换应用之平移

In [13]:
#代码举例1----平移
#平移的变换矩阵为[1 0  dx,0  1  dy],dx,dy是x,y方向平移的距离。结合上面dst的计算公式自己推到M的由来。

import cv2
import numpy as np
img=cv2.imread("imgs\\lena.bmp")
height,width=img.shape[:2]
x=100
y=200
M = np.float32([[1, 0, x], [0, 1, y]])
move=cv2.warpAffine(img,M,(width,height),cv2.INTER_CUBIC,cv2.BORDER_REFLECT,borderValue=(105,155,15))
cv2.imshow("original",img)
cv2.imshow("move",move)
cv2.waitKey(0)
cv2.destroyAllWindows()


#### 3.8.3.2 仿射变换应用之旋转

1、构造旋转矩阵<br>
 （1）利用旋转的原理构造旋转矩阵（略）<br>
 （2）利用cv2.getRotationMatrix2D(center,angle,scale)<br>
    第一个参数：表示向以哪一点进行旋转？这里就是图像的中心 <br>
    第二个参数：表示我们希望旋转的角度。 <br>
    第三个参数：表示图像旋转后的缩放比例<br>
 2、代码举例

In [15]:
#代码举例
import cv2
img=cv2.imread("imgs\\lena.bmp")
height,width=img.shape[:2]
M=cv2.getRotationMatrix2D((width/2,height/2),45,0.6)
rotate=cv2.warpAffine(img,M,(width,height))
cv2.imshow("original",img)
cv2.imshow("rotation",rotate)
cv2.waitKey()
cv2.destroyAllWindows()

#### 3.8.3.3 仿射变换应用之复杂仿射变换

1、构造旋转矩阵<br>
   retval = cv.getAffineTransform(src, dst)
     src：输入图像的三个点坐标
     dst：输出图像的三个点坐标
     三个点分别对应左上角、右上角、左下角
 2、代码举例

In [18]:
import cv2
import numpy as np
img=cv2.imread("imgs\\lena.bmp")
rows,cols,ch=img.shape
p1=np.float32([[0,0],[cols-1,0],[0,rows-1]])
p2=np.float32([[0,rows*0.55],[cols*0.85,rows*0.25],[cols*0.15,rows*0.7]])
M=cv2.getAffineTransform(p1,p2)
dst=cv2.warpAffine(img,M,(cols,rows))
cv2.imshow("origianl",img)
cv2.imshow("result",dst)

cv2.waitKey()
cv2.destroyAllWindows()

#### 车牌校正例子

In [3]:
import cv2
import numpy as np
img=cv2.imread("imgs\\plate.png")
rows,cols,ch=img.shape
p1=np.float32([[4,11],[195,27],[4,89]])
p2=np.float32([[0,0],[195,0],[0,78]])
M=cv2.getAffineTransform(p1,p2)
dst=cv2.warpAffine(img,M,(cols,rows))
cv2.imshow("origianl",img)
cv2.imshow("result",dst)
cv2.imwrite("imgs\\platewarp.png",dst)
cv2.waitKey()
cv2.destroyAllWindows()

执行上述代码，可以看到车牌歪斜得到了一定程度的校正，但是仍存在透视变形，需要用下面的透视变换进行校正。

### 3.8.4 透视变换
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;上述仿射变换可以将矩形映射成任意平行四边形，各边仍保持平行；而透视变换可以将矩形映射为任意四边形，直线仍保持直线。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;由于不再是平行四边形，需提供四个顶点。透视变换通过函数cv2.warpPerspective()实现，语法为:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;dst = cv.warpPerspective(src, M, dsize[, flags[, borderMode[, borderValue]]])<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;dst：透视后的输出图像，dsize决定输出图像大小<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;src：输入图像<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;M：3*3变换矩阵<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;flags：插值方法，默认为INTER_LINEAR<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;borderMode：边类型，默认为BORDER_CONSTANT<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;borderValue：边界值，默认为0<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;透视变换通过函数cv2.getPerspectiveTransform()来生成转换矩阵，需输入输入图像和输出图像的四个顶点的坐标。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;cv2.getPerspectiveTransform(src,dst)<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    src：输入图像的四个点坐标<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    dst：输出图像的四个点坐标<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    四个点的位置应该于映射的期望相对应。<br>
    

In [21]:
import cv2
import numpy as np
img=cv2.imread("imgs\\platewarp.png")
rows,cols,ch=img.shape
p2=np.float32([[0,0],[195,0],[0,78],[195,78]])
p1=np.float32([[0,0],[195,0],[0,78],[191,84]])
M=cv2.getPerspectiveTransform(p1,p2)
dst=cv2.warpPerspective(img,M,(cols,rows))
cv2.imshow("origianl",img)
cv2.imshow("result",dst)
cv2.waitKey()
cv2.destroyAllWindows()

思考题：（1）如何自动选择的旋转的参考点，而不是手工选择？<br>
(2)如果畸变是非线性怎么办？<br>

### 3.9 重映射(remap)
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;把一幅图像内的像素点放置到另一幅图像的指定位置，这个过程叫重映射。opencv提供了remap函数实现此功能：<br>
  remap 的语法格式如下：<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; cv2.remap(src, map1, map2, interpolation, dst=None, borderMode=None, borderValue=None)<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; src是源图像数据，map1是用于插值的X坐标，map2是用于插值的Y坐标，interpolation是采用的插值算法。<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 支持的插值算法有：<br>
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; INTER_NEAREST - 最近邻插值<br>
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; INTER_LINEAR – 双线性插值（默认值）<br>
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; INTER_CUBIC – 双三次样条插值（逾4×4像素邻域内的双三次插值）<br>
  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; INTER_LANCZOS4 -Lanczos插值（逾8×8像素邻域的Lanczos插值）<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; borderMode，边界模式，有默认值BORDER_CONSTANT，表示目标图像中“离群点（outliers）”的像素值不会被此函数修改。<br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; borderValue，当有常数边界时使用的值，其有默认值Scalar( )，即默认值为0。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;重映射通过修改像素点的位置得到-幅新图像。在构建新图像时，需要确定新图像中每个像素点在原始图像中的位置。因此，映射函数的作用是查找新图像像素在原始图像内的位置。该过程是将新图像像素映射到原始图像的过程，因此被称为反向映射。在函数cv2.remap()中，参数mapl和参数map2用来说明反向映射，map1针对的是坐标x，map2针对的是坐标y。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;需要说明的是，mapl和map2的值都是浮点数。因此，目标图像可以映射回一个非整数的值，这意味着目标图像可以“反向映射”到原始图像中两个像素点之间的位置(当然，该位置是不存在像素值的)。这时，可以采用不同的方法实现插值，函数中的interpolation参数可以控制插值方式。正是由于参数map1和参数map2的值是浮点数，所以通过函数cv2.remap()所能实现的映射关系变得更加随意，可以通过自定义映射参数实现不同形式的映射。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;需要注意的是，函数cv2.remap0中参数map1指代的是像素点所在位置的列号，参数map2指代的是像素点所在位置的行号。例如，我们想将目标图像(映射结果图像)中某个点A映射为原始图像内处于第0行第3列上的像素点B,那么需要将A点所对应的参数map1对应位置上的值设为3,参数map2对应位置上的值设为0。所以通常情况下，我们将mapl写为mapx,并且将map2写成mapy，以方便理解。<br>
说明：remap函数的核心是自己设计算法构建原图像和目标图像二者之间像素位置的对应关系，计算出mapx和mapy是核心。<br>
下面通过车牌非线性校正的例子介绍如何构建映射关系。

图像几何变形的关键是要找到原始图像与目标图像之间的投影关系模型，一旦找到该模型则可以利用$x_{src}=s(x_{dst},y_{dst})$和$y_{src}=t(x_{dst},y_{dst})$(此处使用的是向后映射的关系）实现投影变换。然而，实际应用中找到这样的变换模型并不是一件简单的事儿，因为实际的畸变并不是用几个简单的解析式子可以表达出来的，即使能表示成几个表达式，其中的参数确定也是一件非常难求解的问题。<br>
 此处我们假设变换关系已知，仅需要确定相关系数。模型如下:<br>
 <center>$\left\{ \begin{array} {cccc}
x_{src}=s(x_{dst},y_{dst})=c_1x_{dst}+c_2y_{dst}+c_3x_{dst}y_{dst}+c_4\\
y_{src}=t(x_{dst},y_{dst})=c_5x_{dst}+c_6y_{dst}+c_7x_{dst}y_{dst}+c_8
\end{array} \right.$</center><br>
问题的核心转换为如何确定$c_1~c_8$这几个参数。确定方法：选择几个基准点，已知这几个基准点对应到目标图像中的位置坐标，将其带入上式，解方程组得到系数。上述方程有8各未知数，可以选择四对参考点。以车牌校正为例，选择4个角上的点作为参考点。如下图所示:<br>
<img src="imgs/platedemo.png" width="40%"><br>
以校正下图为例。<br>
<img src="imgs/车牌非线性校正原图.png" width="40%"><br>
中国车牌的长宽比大约为3.1:1,假定车牌的长度定为310像素，宽度为100像素，4对参考点坐标的对应关系如下<br>
$(x_{src1},y_{src1})=（13，55）--->(x_{dst1},y_{dst1})=（0，0）$<br>
$(x_{src2},y_{src2})=(205,10)--->(x_{dst2},y_{dst2})=(309,0)$<br>
$(x_{src3},y_{src3})=(201,64)--->(x_{dst3},y_{dst3})=(309,99)$<br>
$(x_{src4},y_{src4})=(16,138)--->(x_{dst4},y_{dst4})=(0,99)$<br>
将以上参考点坐标带入上式，得到以下结果<br>
<center>$\left\{ \begin{array} {cccc}
x_{src1}=s(x_{dst1},y_{dst1})=c_1x_{dst1}+c_2y_{dst1}+c_3x_{dst1}y_{dst1}+c_4\\
x_{src2}=s(x_{dst2},y_{dst2})=c_1x_{dst2}+c_2y_{dst2}+c_3x_{dst2}y_{dst2}+c_4\\
x_{src3}=s(x_{dst3},y_{dst3})=c_1x_{dst3}+c_2y_{dst3}+c_3x_{dst3}y_{dst3}+c_4\\
x_{src4}=s(x_{dst4},y_{dst4})=c_1x_{dst4}+c_2y_{dst4}+c_3x_{dst4}y_{dst4}+c_4
\end{array} \right.$</center><br>
改写成矩阵形式为：<br>
<center>$\left[ \begin{array} {cccc}
x_{src1}\\
x_{src2}\\
x_{src3}\\
x_{src4}
\end{array} \right]=\left[ \begin{array} {cccc}
x_{dst1} &y_{dst1} &x_{dst1}y_{dst1} &1\\
x_{dst2} &y_{dst2} &x_{dst2}y_{dst2} &1\\
x_{dst3} &y_{dst3} &x_{dst3}y_{dst3} &1\\
x_{dst4} &y_{dst4} &x_{dst4}y_{dst4} &1
\end{array} \right] \left[ \begin{array} {cccc}
c_1\\
c_2\\
c_3\\
c_4
\end{array} \right]
$</center><br>
<center>$X_{src}=X_{dst}C_{1-4}$</center>
<center>$C_{1-4}=X_{dst}^{-1}X_{src}$</center>
用同样的方式可以求得：<br>
<center>$C_{5-8}=Y_{dst}^{-1}Y_{src}$</center>

 作业：编程实现上述畸变校正方法，将上述车牌校正到矩形。